In [ ]:
import numpy as np
import pandas as pd
import math
from concurrent.futures import ThreadPoolExecutor
from sympy import symbols
from sympy.solvers.solveset import nonlinsolve
import time
import logging

number_of_sim = 180
number_of_drive = 1

# 定义常数和材料属性
E = 169e9
rho = 2330
t = 25e-6
eps_0 = 8.85e-12
beta = 4.730041
alpha_n = (np.sin(beta) - np.sinh(beta)) / (np.cosh(beta) - np.cos(beta))
pi_160 =(160 / 180) * math.pi
pi_10 = (10 / 180) * math.pi
phi = np.zeros(number_of_sim)
phi[i] = i / number_of_sim * pi_160 + pi_10



# 定义变量范围
w_t_values = np.arange(5e-6, 20e-6, 1e-6) #16
l_t_values = np.arange(100e-6, 1000e-6, 50e-6) #19
Q_values = np.arange(5000, 50000 , 2500) # 19
V_values = np.arange(1, 10, 1) # 10
d_values = np.arange(2e-6, 10e-6, 1e-6) #9


# electrode_length_values = np.arange(100e-6, 900e-6 , 10e-6) #81
# electrode_width_values = np.arange(5e-6, 30e-6, 1e-6) #26
# w_c_values = np.arange(5e-6, 20e-6, 0.2e-6) #76 
# l_c_values = np.arange(5e-6, 30e-6 , 1e-6) #26



# w_t_values = [8e-6]
# l_t_values = [800e-6]
electrode_length_values = [700e-6]
electrode_width_values = [20e-6]
w_c_values = [10e-6]
l_c_values = [20e-6]
# Q_values = [20000]
# V_values = [6.4]
# d_values = [2e-6]



In [ ]:
# 生成所有参数组合的网格
param_grid = np.array(np.meshgrid(w_t_values, l_t_values, Q_values, V_values, d_values)).T.reshape(-1, 5)

# 存储结果的列表
tempVar = []

# 遍历所有参数组合
for params in param_grid:
    w_t, l_t, Q, V, d = params

    # 假设的计算逻辑和参数
    E = 169e9
    t = 25e-6
    rho = 2330
    beta = 4.730041
    alpha_n = (np.sin(beta) - np.sinh(beta)) / (np.cosh(beta) - np.cos(beta))
    number_of_elements = 2000
    electrode_length = 700e-6  # 示例固定值
    electrode_width = 20e-6  # 示例固定值
    w_c = 10e-6  # 示例固定值
    l_c = 20e-6  # 示例固定值
    
    # 这里是计算逻辑的简化版本
    Mass = rho * (t * w_t * l_t * 1.0 + electrode_length * electrode_width * t + w_c * l_c * t * 2)  # 示例计算质量
    k_t = E * t * ((w_t / l_t) ** 3)  # 示例计算弹性系数
    omega_0 = math.sqrt(k_t / Mass)  # 计算固有频率
    eps_0 = 8.85e-12
    trans_factor = eps_0 * V * electrode_length * t / (d ** 2)
    force_ac = 5e-3 * trans_factor  # 激励力

    # 添加结果
    tempVar.append({
        'w_t': w_t, 'l_t': l_t, 'Q': Q, 'V': V, 'd': d,
        'k_t':k_t, 'k_e':k_e, 'k_t3':k_t3,'k_e3':k_e3, 
        'Mass': Mass, 'omega_0': omega_0, 'force_ac': force_ac
    })
    
print(tempVar)
print(len(tempVar))

In [ ]:
def calculate(params, i):
    m_c = np.zeros((number_of_sim, number_of_drive)) ## motional current
    x, y = symbols('x y', real=True)
    Mass, k_t, k_e, k_t3, k_e3, force_ac = params['Mass'], params['k_t'], params['k_e'], params['k_t3'], params['k_e3']

    try:
        solutions = nonlinsolve([
            -Mass * (x ** 2) * y + (k_t - k_e) * y + (k_t3 - k_e3) * (y ** 3) * 3 / 4 - force_ac * math.cos(phi[i]), 
            c * x * y - force_ac * math.sin(phi[i])
        ], [x, y])

        for sol in solutions:
            if np.abs(sol[0]) > omega_0 / 2 and np.abs(sol[0]) < omega_0 * 3 / 2 and sol[0] > 0:
                freq[i, ii] = sol[0] / (2 * math.pi)
                m_c[i, ii] = sol[1] * freq[i, ii] * 2 * math.pi * trans_factor / 1e-9 
    except Exception as e:
        print(f"An error occurred: {e}")
        
        

In [ ]:
def calculate_for_parameters(w_t, l_t, electrode_length, electrode_width, w_c, l_c, V, d, Q, E, t, rho, beta, alpha_n, numberofsmallelements,index):

    start_time = time.time()
    results = []
    length_x = np.linspace(0, l_t- l_t/2000, 2000)
    modeshape_unnormalized = -(-np.sin(beta * length_x / l_t) + np.sinh(beta * length_x / l_t) +
                                alpha_n * (-np.cos(beta * length_x / l_t) + np.cosh(beta * length_x / l_t)))
    second_derivative = -1 / np.max(modeshape_unnormalized) * (beta ** 2) * (np.sin(beta * length_x / l_t) + np.sinh(beta * length_x / l_t) + alpha_n * (np.cos(beta * length_x / l_t) + np.cosh(beta * length_x / l_t)))
    first_derivative = -1 / np.max(modeshape_unnormalized) * beta * (-np.cos(beta * length_x / l_t) + np.sinh(beta * length_x / l_t) + alpha_n * (np.sin(beta * length_x / l_t) + np.cosh(beta * length_x / l_t)))

    modeshape1 = modeshape_unnormalized / np.max(modeshape_unnormalized)
    
    dx = length_x[1] - length_x[0]
    m_coef_b = np.sum(modeshape1 ** 2 * dx / l_t)
    k_coef_b = np.sum(second_derivative ** 2 * dx / l_t) 
    k_coef_b3 = np.sum(first_derivative ** 2 * dx / l_t)

    k_tt = k_coef_b / 12 * E * t * ((w_t / l_t) ** 3)
    k_t3 = k_coef_b3 * E * t * w_t / (l_t ** 3)
    k_t = k_tt
    Mass = rho * (t * w_t * l_t * m_coef_b + electrode_length * electrode_width * t + w_c * l_c * t *2)
    omega_0 = math.sqrt(k_t / Mass)
    freq_0 = omega_0 / (2 * math.pi)
    eps_0 = 8.85e-12
    trans_factor = eps_0 * V * electrode_length * t / (d ** 2)
    k_e = 2 * trans_factor * V / d
    k_e3 = 4 * trans_factor * V / (d ** 3)      
    c = math.sqrt(Mass * k_t) / Q
    
    number_of_drive = 1
    number_of_sim = 180
    
    freq = np.zeros((number_of_sim, number_of_drive))
    m_c = np.zeros((number_of_sim, number_of_drive))
    
    for ii in range(number_of_drive):
        vac = 5e-3 + 0.1e-3 * ii        
        force_ac = vac * trans_factor
        vac_round = np.round(vac / 1e-3 , 2)
        for i in range(number_of_sim):
            x, y = symbols('x, y', reals=True)
            solutions = nonlinsolve([-Mass * (x ** 2) * y + (k_t - k_e) * y + (k_t3 - k_e3) * (y ** 3) * 3 / 4 - force_ac * math.cos(0), 
                                c * x * y - force_ac * math.sin(0)], [x, y])
            for j in range(len(solutions)):
                sol = solutions.args[j]
                if np.abs(sol[0]) > omega_0 / 2 and np.abs(sol[0]) < omega_0 * 3 / 2 and sol[0] > 0:
                    freq[i, ii] = sol[0] / (2 * math.pi)
                    m_c[i, ii] = sol[1] * freq[i, ii] * 2 * math.pi * trans_factor / 1e-9

    results.append({
        'w_t': w_t, 'l_t': l_t, 'electrode_length': electrode_length, 'electrode_width': electrode_width, 'w_c': w_c,
        'l_c': l_c, 'Q': Q, 'V': V, 'd': d, 'Vac_ground': vac, 'freq': freq, 'm_c': m_c
    })
    end_time = time.time()
    print(f"Thread {index}: End - Duration: {end_time - start_time:.2f} seconds")
    return results

# 多线程执行
with ThreadPoolExecutor(max_workers=4) as executor:
    futures = []
    index =0
    for l_t in l_t_values:
        for w_t in w_t_values:
            index += 1
            future = executor.submit(calculate_for_parameters, w_t, l_t, electrode_length, electrode_width, w_c, l_c, V, d, Q, E, t, rho, beta, alpha_n, numberofsmallelements,index)
            futures.append(future)
    
    results = [f.result() for f in futures]

# 组合结果并保存
final_results = [item for sublist in results for item in sublist]
results_df = pd.DataFrame(final_results)
results_df.to_hdf('filename.h5', 'mydata', mode='w')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.mlab as mlab
from scipy.optimize import fsolve
import math
import scipy.io as spio
from sympy.core.symbol import symbols
from sympy.solvers.solveset import nonlinsolve
import pandas as pd


results = []

numberofsmallelements = 2000
length_x = np.zeros(numberofsmallelements)
modeshape_unnormalized = np.zeros(numberofsmallelements)
second_derivative = np.zeros(numberofsmallelements)
first_derivative = np.zeros(numberofsmallelements)

hasresult = 0
Noresult = 0

for l_t in l_t_values:
    length_x = np.linspace(0, l_t- l_t/2000, 2000)
    modeshape_unnormalized = -(-np.sin(beta * length_x / l_t) + np.sinh(beta * length_x / l_t) +
                                alpha_n * (-np.cos(beta * length_x / l_t) + np.cosh(beta * length_x / l_t)))
    second_derivative = -1 / np.max(modeshape_unnormalized) * (beta ** 2) * (np.sin(beta * length_x / l_t) + np.sinh(beta * length_x / l_t) + alpha_n * (np.cos(beta * length_x / l_t) + np.cosh(beta * length_x / l_t)))
    first_derivative = -1 / np.max(modeshape_unnormalized) * beta * (-np.cos(beta * length_x / l_t) + np.sinh(beta * length_x / l_t) + alpha_n * (np.sin(beta * length_x / l_t) + np.cosh(beta * length_x / l_t)))

    modeshape1 = modeshape_unnormalized / np.max(modeshape_unnormalized)
    
    dx = length_x[1] - length_x[0]
    m_coef_b = np.sum(modeshape1 ** 2 * dx / l_t)
    k_coef_b = np.sum(second_derivative ** 2 * dx / l_t) 
    k_coef_b3 = np.sum(first_derivative ** 2 * dx / l_t)
    for w_t in w_t_values:
        k_tt = k_coef_b / 12 * E * t * ((w_t / l_t) ** 3)
        k_t3 = k_coef_b3 * E * t * w_t / (l_t ** 3)
        k_t = k_tt
        Mass = rho * (t * w_t * l_t * m_coef_b + electrode_length * electrode_width * t + w_c * l_c * t *2)
        omega_0 = math.sqrt(k_t / Mass)
        freq_0 = omega_0 / (2 * math.pi)
        for V in V_values:
            for d in d_values:
                eps_0 = 8.85e-12
                trans_factor = eps_0 * V * electrode_length * t  / (d ** 2)
                k_e = 2 * trans_factor * V / d
                k_e3 = 4 * trans_factor * V / (d ** 3)      
                for Q in Q_values:
                    c = math.sqrt(Mass * k_t) / Q
                # 
                    number_of_drive = 1
                    number_of_sim = 180
                    
                    freq = np.zeros((number_of_sim, number_of_drive))
                    m_c = np.zeros((number_of_sim, number_of_drive)) ## motional current
                    
                    
                    for ii in range(number_of_drive):
                        vac = 5e-3 + 0.1e-3 * ii        
                        force_ac = vac * trans_factor
                        vac_round = np.round(vac / 1e-3 , 2)
                        for i in range(number_of_sim):
                            x, y = symbols('x, y', reals=True)
                            try:
                                solutions =  nonlinsolve([-Mass * (x ** 2) * y + (k_t - k_e) * y + (k_t3 - k_e3) * (y ** 3) * 3 / 4 - force_ac * math.cos(phi[i]), 
                                                    c * x * y - force_ac * math.sin(phi[i])], 
                                                    [x, y])
                                for j in range(len(solutions)):
                                    sol = solutions.args[j]
                                    if np.abs(sol[0]) > omega_0 / 2 and np.abs(sol[0]) < omega_0 * 3 / 2 and sol[0] > 0:
                                        freq[i, ii] = sol[0] / (2 * math.pi)
                                        m_c[i, ii] = sol[1] * freq[i, ii] * 2 * math.pi * trans_factor / 1e-9 
                                    else:
                                        pass    
                            except:
                                pass
                    results.append(
                        {
                            'w_t': w_t,
                            'l_t': l_t,
                            'electrode_length': electrode_length,
                            'electrode_width': electrode_width,
                            'w_c': w_c,
                            'l_c': l_c,
                            'Q': Q,
                            'V': V,
                            'd': d,
                            'Vac_ground': vac,
                            'freq': freq[:, 0],
                            'm_c': m_c[:, 0]
                        }
                    )
                                            
                            
results_df = pd.DataFrame(results, columns=['w_t', 'l_t', 'electrode_length', 'electrode_width', 'w_c', 'l_c', 'Q', 'V', 'd', 'Vac_ground' ,'freq', 'm_c'])
results_df.to_hdf('filename.h5', 'mydata', mode='w')
                            

In [ ]:
mpl_fig = plt.figure()
ax = mpl_fig.add_subplot(111)
data_linewidth = 2

line1, = ax.plot(freq[:,0], m_c[:,0], lw=data_linewidth, color = 'tab:red', label = label_fig[0])
line2, = ax.plot(freq[:,1], m_c[:,1], lw=data_linewidth, color = 'tab:blue', label = label_fig[1])
line3, = ax.plot(freq[:,2], m_c[:,2], lw=data_linewidth, color = 'tab:green', label = label_fig[2])
line4, = ax.plot(freq[:,3], m_c[:,3], lw=data_linewidth, color = 'tab:orange', label = label_fig[3])
line5, = ax.plot(freq[:,4], m_c[:,4], lw=data_linewidth, color = 'tab:purple', label = label_fig[4])

#ax.plot(Time_stamp, model(Time_stamp, *popt1), lw=data_linewidth, color = 'r-')

#ax.set_title("My Plot Title")
ax.set_xlabel("Frequency (Hz)")
ax.set_ylabel("Motional current (nA)")

ax.tick_params(which='both', direction='in')
ax.grid(linestyle='--', linewidth=0.5)


ax.legend()
